잠재디리클레할당(LDA, Latent Dirichlet Allocation)

##### 환경준비

##### 1)  sklearn 활용

In [ ]:
#20newgroups 데이터를 전처리하여 텍스트를 클린(clean)하고 토큰화된 형태로 반환

nltk.download('stopwords')
from nltk.corpus import stopwords
import pandas as pd
from sklearn.datasets import fetch_20newsgroups

def get_news(apply_split=True) :
  dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes')) #fetch_20newsgroups함수를 사용하여 20newsgroups데이터 다운로드
  documents = dataset.data

  news_df = pd.DataFrame({'document':documents}) #다운로드한 데이터를 Pandas DataFrame으로 변환
  news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ") #특수문자 제거
  news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3])) #길이가 3보다 작은 단어 제거
  news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower()) #모든 단어를 소문자로 변환
  tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split()) #토큰화. tokenized_doc는 토큰화한 단어들의 리스트가 담긴 Series

  stop_words = stopwords.words('english') #영어 불용어 리스트

  if apply_split :
    return tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
  else :
    return tokenized_doc.apply(lambda x: ' '.join([item for item in x if item not in stop_words]))
  
  #apply_split=True: 각 문서에서 불용어를 제거한 후 토큰화된 단어 목록을 반환
  #apply_split=False: 각 문서에서 블용어를 제거한 후 단어들을 다시 결합하여 하나의 문서로 반환

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tjoeun\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def my_tokenizer(text):
    return text.split()

tokenized_docs = get_news(False) #전처리된 텍스트 데이터를 단어로 토큰화하여 각각의 문서를 리스트로 저장

##### gensim 활용

In [ ]:
#Gensim을 사용하여 LDA(Latent Dirichlet Allocation) 토픽 모델링

from gensim import corpora
from gensim.models import LdaModel, TfidfModel

tokenized_docs = get_news() #get_news() 함수를 사용하여 전처리된 텍스트 데이터 가져오기
id2word = corpora.Dictionary(tokenized_docs) #각 문서를 단어의 출현 빈도로 변환
corpus_TDM = [id2word.doc2bow(doc) for doc in tokenized_docs]
tfidf = TfidfModel(corpus_TDM) #단어의 중요도를 계산하는 TF-IDF모델 생성
corpus_TFIDF = tfidf[corpus_TDM]

n = 20 #토픽의 개수=20
lda = LdaModel(corpus=corpus_TFIDF,
                    id2word=id2word,
                    num_topics=n,
                    random_state=100)
for t in lda.print_topics() : #학습된 LDA모델의 각 토픽에 속하는 단어들 출력
  print(t)

(0, '0.001*"stats" + 0.001*"please," + 0.001*"sports" + 0.001*"henrik]" + 0.001*"posts" + 0.001*"played" + 0.000*"ford" + 0.000*"deleted" + 0.000*"safety" + 0.000*"readers"')
(1, '0.000*"tiff" + 0.000*"animation" + 0.000*"telnet" + 0.000*"floptical" + 0.000*"graphic" + 0.000*"orchid" + 0.000*"book?" + 0.000*"offense" + 0.000*"ethernet" + 0.000*"problems."')
(2, '0.001*"values" + 0.000*"cursor" + 0.000*"maine" + 0.000*"temp" + 0.000*"strikes" + 0.000*"meaningless" + 0.000*"too," + 0.000*"captain" + 0.000*"number?" + 0.000*"adjective"')
(3, '0.001*"port" + 0.001*"(deletion)" + 0.000*"borland" + 0.000*"cancer" + 0.000*"port." + 0.000*"responses" + 0.000*"where?" + 0.000*"linux" + 0.000*"luck." + 0.000*"grateful"')
(4, '0.001*"arab" + 0.001*"players" + 0.001*"for." + 0.001*"palestinian" + 0.001*"arabs" + 0.001*"------" + 0.001*"clemens" + 0.001*"phillies" + 0.000*"jews" + 0.000*"owners"')
(5, '0.003*"would" + 0.003*"know" + 0.003*"like" + 0.002*"people" + 0.002*"think" + 0.002*"could" + 0.

In [ ]:
#각 단어의 중요도를 나타내는 TF-IDF 모델 생성

corpus_TDM = [id2word.doc2bow(doc) for doc in tokenized_docs] #각 문서를 단어의 출현 빈도로 변환
tfidf = TfidfModel(corpus_TDM) #TF-IDF 모델 생성. DTM을 사용하여 각 단어의 TF-IDF 가중치 계산
corpus_TFIDF = tfidf[corpus_TDM] #TF-IDF 모델을 사용하여 TF-IDF가중치 계산 -> 문서-단어 행렬(corpus_TFIDF) 생성

In [ ]:
#pyLDAvis(=토픽 모델링 결과를 시각적으로 탐색할 수 있는 도구)를 사용하여 LDA모델의 결과를 시각화 => 토픽들 간의 상호작용 확인
import pyLDAvis
import pyLDAvis.gensim

pyLDAvis.enable_notebook() #Jupyter Notebook에서 pyLDAvis를 사용할 수 있도록 활성화
vis = pyLDAvis.gensim.prepare(lda, corpus_TFIDF, id2word, mds='PCoA') # id2word는 딕셔너리임
pyLDAvis.display(vis)

#lda: 학습된 LDA모델
#corpus_TFIDF: TF-IDF로 가중치가 적용된 문서-단어 행렬
#id2word: 단어 ID와 해당 단어를 매핑하는 딕셔너리
#mds 매개변수는 다차원 척도법(Multidimensional Scaling)을 설정하는 데 사용